Reference: https://www.pythonforengineers.com/audio-and-digital-signal-processingdsp-in-python/

人耳你能听到的频率范围:
https://blog.csdn.net/wangjianzhongfj/article/details/72860919
1.  20Hz--60Hz部分
这一段提升能给音乐强有力的感觉，给人很响的感觉，如雷声。是音乐中强劲有力的感觉。如果提升过高，则又会混浊不清，造成清晰度不佳，特别是低频响应差和低频过重的音响设备。

2.  60Hz--250Hz部分
这段是音乐的低频结构，它们包含了节奏部分的基础音，包括基音、节奏音的主音。它和高中音的比例构成了音色结构的平衡特性。提升这一段可使声音丰满，过度提升会发出隆隆声。衰减这两段会使声音单薄。

3.  250Hz--2KHz部分
这段包含了大多数乐器的低频谐波，如果提升过多会使声音像电话里的声音。如把600Hz和1kHz过度提升会使声音像喇叭的声音。如把3kHz提升过多会掩蔽说话的识别音，即口齿不清，并使唇音“mbv”难以分辨。如把1kHz和3kHz过分提升会使声音具有金属感。由于人耳对这一频段比较敏感，通常不调节这一段，过分提升这一段会使听觉疲劳。

4.  2KHz--4kHz部分
这段频率属中频，如果提升得过高会掩盖说话的识别音，尤其是3kHz提升过高，会引起听觉疲劳。

5.  4kHz--5KHz部分
这是具有临场感的频段，它影响语言和乐器等声音的清晰度。提升这一频段，使人感觉声源与听者的距离显得稍近了一些；衰减5kHz，就会使声音的距离感变远；如果在5kHz左右提出升6dB，则会使整个混合声音的声功率提升3dB。

6.  6kHz--16kHz部分
这一频段控制着音色的明亮度，宏亮度和清晰度。一般来说提升这几段使声音宏亮，但不清晰，不可能会引起齿音过重，衰减时声音变得清晰，但声音不宏亮。”

人说话的频率范围：85HZ-8kHZ

基频： 主导频率;
基频整数倍的频率分量称为谐波，也称泛音
基音频率范围是：
- 童声高音频率范围为260-880Hz，低音频率范围为196-700Hz，
- 女声高音频率范围为220-1.1KHz,低音频率范围为200-700Hz，
- 男声高音频率范围为160-523Hz低音频率范围为80-358Hz。

发音原理：


In [ ]:
import os
import wave
import struct

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
frequency = 1000 # Hz
noisy_freq = 50 # Hz
sample_rate = 48000

duration = 1 # unit s
amplitude = 16000

num_samples = sample_rate * duration

file = ("/home/leliang/project/baidu/tmp/leliang/suppress_noise_%(freq)dHz.wav" % {'freq':frequency})

if os.path.exists(file):
    os.remove(file)

delta_t = 1.0 / sample_rate

# 生成音频信号
sine_wave = [np.sin(2 * np.pi * frequency * (x * delta_t)) for x in range(num_samples)]
sine_noise = [np.sin(2 * np.pi * noisy_freq * (x * delta_t)) for x in range(num_samples)]

sine_wave = np.array(sine_wave)
sine_noise = np.array(sine_noise)

combined_signal = sine_wave + sine_noise

plt.subplot(3, 1, 1)
plt.title("sin 1000Hz")
plt.subplots_adjust(hspace=.5)
samples_5p = int(0.05 * sample_rate) #0.05s的采样数
plt.plot(sine_wave[:samples_5p])

plt.subplot(3, 1, 2)
plt.title("sin 50Hz")
plt.subplots_adjust(hspace=.8)
samples_5p = int(0.05 * sample_rate) #0.05s的采样数
plt.plot(sine_noise[:samples_5p])

plt.subplot(3, 1, 3)
plt.title("combined")
plt.subplots_adjust(hspace=.8)
samples_5p = int(0.05 * sample_rate) #0.05s的采样数
plt.plot(combined_signal[:samples_5p])

print("wav mean: ", np.sum(sine_wave))
print("noise mean: ", np.sum(sine_noise))
print("combined mean: ", np.sum(combined_signal))

In [ ]:
if os.path.exists(file):
    os.remove(file)

nframes=num_samples
comptype='NONE'
compname='not compressed'
nchannels=1
sample_width=2 # byte(16bits)

wave_file=wave.open(file, 'w')
wave_file.setparams((nchannels, sample_width, int(sample_rate), nframes, comptype, compname))

for value in combined_signal:
    wave_file.writeframes(struct.pack('h', int(value * amplitude)))
    
wave_file.close()

分析合成后信号

In [ ]:
wave_file = wave.open(file, 'r')

nframes = wave_file.getnframes()
framerate = wave_file.getframerate()
sampwidth = wave_file.getsampwidth()

data = wave_file.readframes(nframes)
data_s = struct.unpack('{n}h'.format(n=nframes), data)
ys = np.array(data_s) / amplitude

plt.title("combined")
samples_5p = int(0.05 * sample_rate) #0.05s的采样数
plt.plot(ys[:samples_5p])

wave_file.close()

In [ ]:
data_fft = np.fft.fft(combined_signal)
#freqs = (np.abs(data_fft[:len(data_fft)]))
freqs = data_fft
print(freqs[0:2])

plt.plot(freqs)
plt.title('Before filtering: Will have main signal(1000Hz) + noise(50Hz)')
plt.xlim(0, 1500)

In [ ]:
data_ifft = np.fft.ifft(data_fft)

data_ifft_part = data_ifft[0:int(0.05 * sample_rate)]
plt.plot(data_ifft_part)
plt.title('Before filtering: Will have main signal(1000Hz) + noise(50Hz)')
#plt.xlim(-1, 1500)

print("sum data_fft:", np.sum(data_fft))

In [ ]:
#滤波去噪(带通滤波器) 假定有效频率的范围是500->1500
start_freq = 500 #Hz
end_freq = 1500 #Hz

index_freq = 0
filtered_freqs = []
for f in freqs:
    if index_freq > 500 and index_freq < 1500:
        #if f > 1e-20: # 过滤掉比较小的值
        filtered_freqs.append(f)
        #else:
        #    filtered_freqs.append(0) # 小值频率过滤掉
    elif index_freq > 46500 and index_freq < 47500:
        #if f > 1e-20: # 过滤掉比较小的值
        filtered_freqs.append(f)
        #else:
        #    filtered_freqs.append(0) # 小值频率过滤掉
    else:
        filtered_freqs.append(0) # 带外频率清空
    index_freq += 1
    
plt.plot(filtered_freqs)
plt.title('after filtering')
#plt.xlim(0, 1500)


In [ ]:
recovered_signal = np.fft.ifft(filtered_freqs)
print(recovered_signal[0:20])
recovered_signal = recovered_signal.real #? ??
#recovered_signal = np.fabs(recovered_signal)

rec_max, rec_min = recovered_signal.max(), recovered_signal.min()
rec_max = max(np.abs(rec_max), np.abs(rec_min))
print(rec_max)

#recovered_signal = recovered_signal / rec_max

plt.title("recovered_signal")
plt.subplots_adjust(hspace=.8)
samples_5p = int(0.01 * sample_rate) #0.05s的采样数
plt.plot(recovered_signal[:samples_5p])

print("sum(filtered_freqs):", np.sum(filtered_freqs))
print(np.sum(recovered_signal))

In [ ]:
plt.title("sin 1000Hz")
plt.subplots_adjust(hspace=.5)
samples_5p = int(0.01 * sample_rate) #0.05s的采样数
plt.plot(sine_wave[:samples_5p])

In [ ]:
array_len = len(data_fft)

freq_window = np.zeros(array_len)
freq_window[500:1500] = 1
freq_window[46500:47500] = 1

filtered_freqs = data_fft * freq_window

recovered_signal = np.fft.ifft(filtered_freqs)
print(recovered_signal[0:20])
recovered_signal = recovered_signal.real #? ??
#recovered_signal = np.fabs(recovered_signal)

rec_max, rec_min = recovered_signal.max(), recovered_signal.min()
rec_max = max(np.abs(rec_max), np.abs(rec_min))
print(rec_max)

#recovered_signal = recovered_signal / rec_max

plt.title("recovered_signal")
plt.subplots_adjust(hspace=.8)
samples_5p = int(0.01 * sample_rate) #0.05s的采样数
plt.plot(recovered_signal[:samples_5p])

print("sum(filtered_freqs):", np.sum(filtered_freqs))
print(np.sum(recovered_signal))

<p> 噪声 三种性质
    
* 分布（Distribution）：随机信号的分布值的是他们可能产生的值以及产生这些值的概率。 例如，在均匀噪声中，信号的值可以在[-1,1]的范围中，产生这些值的概率都是一样的。 而高斯噪声（ Gaussian Noise ）则不同，它的值可以从负无穷到正无穷，但是越接近0 的值出现的概率越高，它的概率分布图是一个高斯分布曲线（也叫正态分布曲线或钟型曲线）
* 表示信号的不同值之间是独立的或是有依赖关系。在UU噪声中， 所有的值都是独立的。而布朗噪声（ Brownian noise ）则不同，它的每个值都是前一时刻 信号的值再加上一个随机的“步值（step）”。因此，如果某一个时刻信号的值比较大， 我们可以预测它接下来依然会保持比较大的值，反之亦然。
* 功率和频率之间的关系：在UU噪声中，所有频率成分的功率看起来是均匀分布的，也就是说信号的功率 被平均分布到的所有的频率成分上。而粉红噪声（ Pink Noise ）则不同， 它的功率与频率成反比例关系，也就是说对于频率为 f 的成分来说，它的功率为 1/f

<p> <b>频谱积分:</b> 通过对频谱进行积分（ integrated spectrum ）来更清晰的观察到频率和 功率之间的关系。频谱积分就是计算频率从0到f上的功率的累加和。

In [ ]:
import thinkdsp
import thinkplot

In [ ]:
#不相关噪声， UU噪声， 白噪声：灯发出所有频率的可见光 并且功率都一样，那么灯光的颜色应该是白色的。
signal = thinkdsp.UncorrelatedUniformNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.make_audio()

In [ ]:
#UU噪声 时域信号
thinkplot.config(xlim=[0,0.1])
wave.plot(linewidth=0.5)


In [ ]:
#UU噪声 频谱
spectrum = wave.make_spectrum()
spectrum.plot_power(linewidth=0.5)

In [ ]:
#UU噪声 频谱积分

integ = spectrum.make_integrated_spectrum()
integ.plot_power()

#积分后的频谱是一条直线，表示所有频率成分的平均功率是一个常值

In [ ]:
#布朗噪声, 红噪声
signal = thinkdsp.BrownianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.make_audio()

In [ ]:
#布朗噪声 时域信号
wave.plot()

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power(linewidth=1, alpha=0.5)

#在线性刻度下，布朗噪声的频谱图的几乎所有的频率成分都在最是低频，高频分量几乎不可见

In [ ]:
spectrum.plot_power(linewidth=1, alpha=0.5)
thinkplot.config(xscale='log', yscale='log')

#频率和功率的关系虽然也呈现噪声的特点，但是又有一定的线性规律
# P = K/f^2 => red noise

In [ ]:
t = spectrum.estimate_slope()
print(t)

In [ ]:
#pink noise 分红噪声
#P = K / f^^b  => b = 0:white noise, b=2:red noise, 0<b<2:pink noise
signal = thinkdsp.PinkNoise(amp=1.0, beta=1.0)
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.make_audio()


In [ ]:
wave.plot(linewidth=0.5)

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot(linewidth=0.5)

In [ ]:
spectrum.plot_power(linewidth=1, alpha=0.5)
thinkplot.config(xscale='log', yscale='log')

In [ ]:
#高斯噪声
signal = thinkdsp.UncorrelatedGaussianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
spectrum = wave.make_spectrum()
spectrum.plot(linewidth=0.5)

In [ ]:
import thinkstats2
# 正态概率图 正态概率图中，如果数据与直线拟合的很好，则说明数据是服从高斯分布的
thinkstats2.NormalProbabilityPlot(spectrum.real)
plt.show()
thinkstats2.NormalProbabilityPlot(spectrum.imag)

In [ ]:
help(plt.plot)